In [6]:
#@title Environment setup
#@markdown * Clone CASA Github https://github.com/lopentu/casa

import getpass, os
username = getpass.getpass("Github username: ")
password = getpass.getpass("Github password: ")
os.environ["CRED"] = f"{username}:{password}"
!git clone https://${CRED}@github.com/lopentu/casa
!cd casa && git checkout seantyh
del username, password

#@markdown * install CASA dependencies
!pip install -r /content/casa/requirements.txt

#@markdown * Install CKIPTagger
!pip install -U ckiptagger[tfgpu,gdown]
from ckiptagger import data_utils
!mkdir -p ckiptagger
!cp "/content/drive/MyDrive/Files/CKIPTagger/data.zip" /content/ckiptagger/data.zip
!cd /content/ckiptagger/ && unzip data.zip
# data_utils.download_data_url("./ckiptagger")

#@markdown * Mount google drive
from google.colab import drive
drive.mount('/content/drive')

#@markdown * Build symbolic link
!rm -r /content/casa/data
!ln -s "/content/drive/Shareddrives/LOPE/產學合作/中華電信/2020-2021/指向情緒案/data" /content/casa/data


KeyboardInterrupt: ignored

In [4]:
!rm -r /content/casa/data
!ln -s "/content/drive/Shareddrives/LOPE/產學合作/中華電信/2020-2021/指向情緒案/data" /content/casa/data

rm: cannot remove '/content/casa/data': No such file or directory


## Load data

In [2]:
%load_ext autoreload
%autoreload 2
import sys
sys.path = ["/content/casa/src"] + sys.path

In [5]:
import pickle
import casa
raw_pkl_fname = "raw-data-cht-202010.pkl"
with open(f"casa/data/{raw_pkl_fname}", "rb") as fin:
  threads = pickle.load(fin)

In [6]:
print(len(threads))
threads[0].print_thread()

7772
Main: <Opinion [Dcard] 東吳大學: 最近各家電信一直打電話傳簡訊詢問要不要續...>
-- <Opinion [Dcard] 吳鳳科技大學: 續>
-- <Opinion [Dcard] 東吳大學: B1 怎麼說>
-- <Opinion [Dcard] 吳鳳科技大學: 5g現在還不穩定 現在辦5g繳5g的錢只...>
-- <Opinion [Dcard] 東吳大學: B3 所以也不建議現在就換5g手機囉？>
-- <Opinion [Dcard] 吳鳳科技大學: 換5g手機可以 但是5g網路可以再等等 ...>
-- <Opinion [Dcard] 東吳大學: B5 好的~>
-- <Opinion [Dcard] 吳鳳科技大學: 我自己 還有家人和同學 都是續30個月 ...>
-- <Opinion [Dcard] 國立東華大學: 等1111>
-- <Opinion [Dcard] 東吳大學: B8 1111是手機方案到期日嗎😂>
-- <Opinion [Dcard] 國立東華大學: B9各大電信優惠日>


In [7]:
from casa.eda import CkipSubmitProcessor, CkipRetrieveProcessor, CkipProxy
from ckiptagger import construct_dictionary, WS, POS
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
ws = WS("./ckiptagger/data", disable_cuda=False)
pos = POS("./ckiptagger/data", disable_cuda=False)
ckip = CkipProxy(ws, pos)
submitter = CkipSubmitProcessor(ckip)
retrieval = CkipRetrieveProcessor(ckip)

In [10]:
## Use GPU if possible, decoding is slow.
from tqdm.auto import tqdm
for thread_x in tqdm(threads):
    thread_x.process(submitter)

In [11]:
ckip.process()

In [12]:
for thread_x in tqdm(threads):
    thread_x.process(retrieval)

In [37]:
len(ckip.done), len(ckip.buf)

(63050, 63050)

In [24]:
def check_seg(thread_x):
  if thread_x.main:
    print("/".join(thread_x.main.text_tokens))
    print(thread_x.main.text)

  if thread_x.replies:
    print("/".join(thread_x.replies[0].text_tokens))
    print(thread_x.replies[0].text)

In [34]:
check_seg(threads[1250])

這/188/方案/一點/都/沒/快閃/呀/，/而是/原本/就/有/，/ /原本/是/到/9/30/號/還/加/抽耳機/咧/，/ /中秋節/的/149月/租/專案/到期/過後/，/又/推出/一模一樣/~/如果/你/是/要/打電話/，/就/辦/138/上網/10/G/用完/降速/成/12/M/，/電話/還可以/打/100/分鐘/。/ P/./S/./我/個人/是/辦/月租/149/，/上網/21/M/吃/到/飽/，/電話/30/分鐘
這188方案一點都沒快閃呀，而是原本就有， 原本是到9/30號還加抽耳機咧， 中秋節的149月租專案到期過後，又推出一模一樣~如果你是要打電話，就辦138上網10G用完降速成12M，電話還可以打100分鐘。 P.S.我個人是辦月租149，上網21M吃到飽，電話30分鐘


## Save the output

In [32]:
import pickle
!mkdir -p /content/casa/data/eda/ckip
ckip_pkl_fname = raw_pkl_fname.replace(".pkl", ".ckip.pkl")
with open(f"/content/casa/data/eda/ckip/{ckip_pkl_fname}", "wb") as fout:
    pickle.dump(threads, fout)

In [13]:
#@title Code snippet when editing and commiting back to Github

!git config --global user.email "seantyh@gmail.com"
!git config --global user.name "seantyh"
!cd casa && git add src/ && git status
!cd casa && git commit -a -m "revision in colab" && git push

On branch seantyh
Your branch is up to date with 'origin/seantyh'.

Changes to be committed:
  (use "git reset HEAD <file>..." to unstage)

	modified:   src/casa/eda/ckip_proc.py
	modified:   src/casa/opinion_thread.py

Untracked files:
  (use "git add <file>..." to include in what will be committed)

	data

[seantyh ffbd059] revision in colab
 2 files changed, 8 insertions(+), 5 deletions(-)
Counting objects: 7, done.
Delta compression using up to 2 threads.
Compressing objects: 100% (6/6), done.
Writing objects: 100% (7/7), 639 bytes | 639.00 KiB/s, done.
Total 7 (delta 5), reused 0 (delta 0)
remote: Resolving deltas: 100% (5/5), completed with 5 local objects.
To https://github.com/lopentu/casa
   ad6e5e6..ffbd059  seantyh -> seantyh


In [ ]:
# !cd casa && git commit -a -m "revision in colab" && git push